In [5]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output, State
from dash import callback_context

import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from crud import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Center(html.B(html.H1('Page By Zachary LaFlair'))),
    html.Hr(),
    #FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    html.Div(className='row',
             style={'display' : 'flex'},
                 children=[
                         html.Button('Cats', id='submit-button-one', n_clicks=0),
                         html.Button(id='submit-button-two', n_clicks=0, children='Dogs'),
                         html.Button(id='submit-button-three', n_clicks=0, children='Water Rescue'),
                         html.Button(id='submit-button-four', n_clicks=0, children='Mountain or Wilderness Rescue'),
                         html.Button(id='submit-button-five', n_clicks=0, children='Disaster Rescue or Individual Tracking'),
                         html.Button(id='submit-button-six', n_clicks=0, children='Clear Filters'),
                 ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 1,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id',"data"), 
    [Input('submit-button-one', 'n_clicks'),
    Input('submit-button-two', 'n_clicks'),
    Input('submit-button-three', 'n_clicks'),
    Input('submit-button-four', 'n_clicks'),
    Input('submit-button-five', 'n_clicks'),
    Input('submit-button-six', 'n_clicks'),
    ])
def update_dashboard(bt1, bt2, bt3, bt4, bt5, bt6):
### FIX ME Add code to filter interactive data table with MongoDB queries
    changed_id = [p['prop_id'] for p in callback_context.triggered][1]
    if 'submit-button-one' in changed_id:
        df = pd.DataFrame.from_records(list(shelter.read({"animal_type" : "Cat"})))
    elif 'submit-button-two' in changed_id:
        df = pd.DataFrame.from_records(list(shelter.read({"animal_type":"Dog"})))
    elif 'submit-button-three' in changed_id:
        df = pd.DataFrame.from_records(list(shelter.read({"animal_type" : "Cat"})))
    elif 'submit-button-four' in changed_id:
        df = pd.DataFrame.from_records(list(shelter.read({"animal_type":"Dog"})))
    elif 'submit-button-five' in changed_id:
        df = pd.DataFrame.from_records(list(shelter.read({"animal_type":"Dog"})))
    elif 'submit-button-six' in changed_id:
        df = pd.DataFrame.from_records(shelter.read({}))
    return df.to_dict('records')
    #return (data,columns)


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    # added a count function to make a bar chart of our animals by type
    dog_count = shelter.count({"animal_type" : "Dog"})  # gets count of dogs
    cat_count = shelter.count({"animal_type" : "Cat"})  # gets count of cats
    all_count = shelter.count({})  # gets total count
    other_count = all_count - (dog_count + cat_count)  # extrapolates 'other' animals IE (total - (dogs + cats))
    
    # returns the graph
    return [
        dcc.Graph(
            figure={
                'data': [
                    {'x': ['Dogs'], 'y': [dog_count], 'type': 'bar', 'name': 'Dogs'},
                    {'x': ['Cats'], 'y': [cat_count], 'type': 'bar', 'name': 'Cats'},
                    {'x': ['Other'], 'y': [other_count], 'type': 'bar', 'name': 'Other'},
                ],
                'layout': {
                    'title': 'Animals By Type'
                }
            }
        )
    ]

    
    
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '800px', 'height': '400px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]
app